## Import required libraries

In [ ]:
import torch
import torchvision
import os
import sys
from tempfile import TemporaryDirectory

ood_path = os.path.abspath('../')
if ood_path not in sys.path:
    sys.path.append(ood_path)
    
from ood_enabler.ood_enabler import OODEnabler
from ood_enabler.model_wrapper.pytorch import PytorchWrapper
from ood_enabler.storage.model_store import ModelStore

## Create Model Store connection to local filesystem

In [ ]:
model_store = ModelStore.from_filesystem()

## Load pretrained ResNet50 model from pytorch and save locally.*
### *(demo purposes only)

## Then reload into memory using FileStorage

In [ ]:
model = torchvision.models.resnet50(pretrained=True)
model_metadata = {'type': 'pytorch', 'arch': 'resnet50', 'ood_thresh_percentile': 20}

with TemporaryDirectory() as tmpdir:
    model_path = os.path.join(tmpdir, 'pytorch_resnet50')
    model_full = torch.jit.script(model)
    model_full.save(model_path)
    
    model = model_store.load(model_metadata, model_path)

## Review model architecure

In [ ]:
model.model

## Enable model with OOD layer

In [ ]:
OODEnabler.ood_enable(model)

## Review new model architecture with embedded OOD Layer

In [ ]:
model.model

## Run `predict` on OOD model to review change in outputs

In [ ]:
from torchvision import transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)

In [ ]:
# OOD model
model.model.eval()
for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    outputs = model.model(inputs)
    break
    

In [ ]:
outputs

# Save OOD enabled model

In [ ]:
model_store.upload(model, '.')